# Benchmark between Matlab and Python implementation of DEPSI spatial unwrapping

We will construct a network with pnts and arcs from matlab data.

The relevant data we are using from MATLAB are:

Inputs:
- arc ambiguities
- source and destination point indices for arcs
- point azimuth and range indices, order matters since it connects to arcs
- reference point id
- ref point azimuth and range

Outputs:
- point ambiguities

Now arc estimation happens in this process, so theoretically pnts and arcs phases should not matter.

In [ ]:
import xarray as xr
import numpy as np
import scipy
import math
from matplotlib import pyplot as plt

# import matplotlib.cm as cm
import matplotlib.colors as plc

from depsi.network import (
    spatial_unwrapping,
)

WAVELENGTH = 0.055465763  # Sentinel-1, in meters

In [ ]:
# Matlab files
file_mat_input = "../../data/marken_tsx_testdata_spatial_integration/marken_tsx_spatial_unwrapping_testdata_input.mat"
file_mat_output = "../../data/marken_tsx_testdata_spatial_integration/marken_tsx_spatial_unwrapping_testdata_output.mat"
data_input = scipy.io.loadmat(file_mat_input)
data_output = scipy.io.loadmat(file_mat_output)

In [ ]:
data_input.keys()

In [ ]:
data_output.keys()

In [ ]:
# Input data
azimuth_idx = data_input["psc_data"][:, 4] # azimuth index
range_idx = data_input["psc_data"][:, 5] # range index
Btemp = data_input["Btemp"].squeeze()
pnt_phase = data_input["psc_data"][:, 6:6+data_input["Btemp"].shape[0]]
pnt_h2ph = data_input["psc_data"][:,(6+data_input["Btemp"].shape[0]):(6+2*data_input["Btemp"].shape[0])]
arc_h2ph = data_input["dpsc_h2ph"] # arc h2ph values
arc_phases = data_input["dpsc_phase"] # arc phase values
arc_ens_coh = data_input["dpsc_ens_coh"].squeeze() # arc ensemble coherence
arc_acheck = data_input["dpsc_acheck"]
Narcs = data_input["Narcs"].squeeze()
idx_source = data_input["dpsc_arcs"][:, 0]-1 # MATLAB to Python index
idx_target = data_input["dpsc_arcs"][:, 1]-1 # MATLAB to Python index

# reference point, this is taken from output data
idx_ref_pnt = data_output["ref_array"][0][0] - 1 # MATLAB to Python index 
refpnt_azimuth = data_output["ref_array"][0][1]
refpnt_range = data_output["ref_array"][0][2]

# Processing parameters
ens_coh_threshold = data_input["ens_coh_threshold"][0][0] # arc selection threshold

In [ ]:
# Assign ambiguity results to original points array
acheck = data_output["acheck"]
idx_nonref_pnts = data_output["non_ref_points"].squeeze()-1
psc_acheck = data_output["psc_acheck_temp"]
psc_acheck[idx_nonref_pnts,:] = np.round(acheck)

In [ ]:
# stm as xr dataset with space and time dimensions
stm = xr.Dataset(
    data_vars={
        "pnt_h2ph": (("space", "time"), pnt_h2ph),
        "pnt_phase": (("space", "time"), pnt_phase),
    },
    coords={
        "time": (("time"), Btemp),
        "space": (("space"), np.arange(azimuth_idx.shape[0])),
        "azimuth": (("space"), azimuth_idx),
        "range": (("space"), range_idx),
    },
)
stm

In [ ]:
# Manually assign a uid
scale = 10 ** (math.floor(math.log10(stm.sizes["space"])) + 1)  # Scale to ensure no overlap
uid = scale * (idx_source + 1) + (idx_target + 1)  # Plus one to avoid zero uid
uid = uid.astype(np.int64)

stm_arcs = xr.Dataset(
    data_vars={
        "h2ph": (("space", "time"), arc_h2ph),
        "ambiguities": (("space", "time"), arc_acheck),
        "ens_coh": (("space"), arc_ens_coh),
        "d_phase": (("space", "time"), arc_phases),
        "uid": (("space"), uid),
    },
    coords={
        "time": (("time"), Btemp),
        "source": (("space"), idx_source),
        "target": (("space"), idx_target),
    },
)
stm_arcs

## Step1: form network and remove arcs with low ensemble coherence

In [ ]:
# x of sources and targets
xx = np.stack(
    [
        stm.isel(space=stm_arcs["source"])["range"].values,
        stm.isel(space=stm_arcs["target"])["range"].values,
    ]
).T
# y of sources and targets
yy = np.stack(
    [
        stm.isel(space=stm_arcs["source"])["azimuth"].values,
        stm.isel(space=stm_arcs["target"])["azimuth"].values,
    ]
).T
# Visualize created arcs
fig, ax = plt.subplots()
cmap = plt.cm.rainbow
norm = plc.Normalize(vmin=0, vmax=1.0)
mean_nmad = np.abs(stm_arcs["ens_coh"].data)
for i in range(stm_arcs.sizes["space"]):
    ax.plot(xx[i], yy[i], color=cmap(norm(mean_nmad[i])), linewidth=0.5)
plt.title("Network STM arcs, colored by ens_coh")
plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax, label="ens_coh")

In [ ]:
Qyy_diag = np.ones(stm_arcs.sizes["space"])

stm_arcs_output, stm_pnts_output = spatial_unwrapping(
    stm, stm_arcs, Qyy_diag, threshold_arc_quality=ens_coh_threshold, idx_refpnt=idx_ref_pnt
)

In [ ]:
plt.imshow(stm_pnts_output["ambiguities"].values, cmap="jet", vmin=-2, vmax=2)

In [ ]:
plt.imshow(acheck, cmap="jet", vmin=-2, vmax=2)